<a href="https://colab.research.google.com/github/jddfrance/ifoodextract/blob/main/ifood_extract_0.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import requests
from bs4 import BeautifulSoup 
import json
import pandas as pd
import numpy as np

In [102]:
def html_trat(input): #tratamento do html pro bs4
    return " ".join(input.split()).replace('> <', '><') 

In [83]:
s = requests.Session() #maintain the same session throught the whole execution
def get_web(url):
  headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Mobile Safari/537.36'}
  texto = s.get(url, headers = headers).text
  return texto

In [173]:
def get_rest(lastitude, longitude, pages = 2):
  restupdate = []
  for page in range(0, pages): #Get pages
    url = f"https://marketplace.ifood.com.br/v1/merchants?latitude={latitude}&longitude={longitude}&channel=IFOOD&size=300&page={page}"
    jload = pd.json_normalize(json.loads(get_web(url))).merchants[0]
    if jload: #check if merchants exists
      restaurantes = (pd.DataFrame.from_dict(jload)
                    .set_index('id', drop= True))
      mercupdate = []
      for item in restaurantes.index: #Get shifts e userRatingCount from merchants
        url = f'https://marketplace.ifood.com.br/v1/merchants/{item}/extra'
        mercupdate.append((pd.json_normalize(json.loads(get_web(url)))
                    .set_index('id', drop = True)
                    [['shifts', 'userRatingCount']]))
    restupdate.append(pd.concat([restaurantes, pd.concat(mercupdate)], axis = 1))

  restaurantes = pd.concat(restupdate) #consolidate data from pages
  prices = []
  for idx, item in restaurantes.iterrows(): #Getting price stats
    link = f"https://www.ifood.com.br/delivery/{item['slug']}"
    soup = BeautifulSoup(html_trat(get_web(link)), 'html.parser')
    jsondata = json.loads(soup.find('script', {'id': "__NEXT_DATA__"}).getText())
    pedidos = pd.json_normalize(jsondata)['props.initialState.restaurant.menu'][0]
    subitem = pd.json_normalize(pedidos)
    itemlist = []
    for index, row in subitem.iterrows():
      df = pd.json_normalize(row['itens'])
      df['Grupo'] = row['name']
      itemlist.append(df)
    if itemlist:  #check if merchant has itens/menu
      pricestats = (pd.concat(itemlist)
          .describe()['unitMinPrice']
          .to_frame()
          .transpose())
      pricestats['id'] = idx
      pricestats.set_index('id', inplace = True)
      prices.append(pricestats)
    else: 
      prices.append(pd.DataFrame([idx],columns = ['id']).set_index('id'))
  return pd.concat([restaurantes, pd.concat(prices)], axis = 1)  
latitude = '-1.3735'
longitude = '-48.4496'
pages = 2
get_rest(latitude, longitude, pages)

https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300&page=0
https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300&page=1
https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300&page=2
https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300&page=3
https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300&page=4
https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300&page=5
https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300&page=6
https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300&page=7
https://marketplace.ifood.com.br/v1/merchants?latitude=-1.3735&longitude=-48.4496&channel=IFOOD&size=300

In [174]:
tst = pd.concat([restaurantes, pd.concat(prices)], axis = 1)
tst.to_csv('ifood-20211128')
tst

,name,resources,slug,userRating,priceRange,mainCategory,features,tags,paymentCodes,minimumOrderValue,deliveryFee,deliveryTime,distance,available,merchantChain,takeoutTime,preparationTime,contextSetup,currency,shifts,userRatingCount,availableForScheduling,nextSchedulingSlotStartTime,nextSchedulingSlotEndTime,count,mean,std,min,25%,50%,75%,max
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ba8c7258-16ab-4f9d-b693-446a29ea1dec,Burger King - Parque Shopping Belém,"[{'type': 'LOGO', 'fileName': 'ba8c7258-16ab-4...",belem-pa/burger-king---parque-shopping-belem-p...,4.50888,CHEAP,"{'code': 'LCH', 'name': 'Lanches'}","[DELIVERY, TRACKING, TRACEABLE, IMMEDIATE_ORDER]","[ADDRESS_PREFORM_TYPE, GUIDED_HELP_TYPE, HAMBU...","[DNR, MPAY, MOVPAY_MC, MC, GPY_ELO, ELOD, GPY_...",20.0,"{'type': 'FIXED', 'value': 6.99, 'originalValu...",37,0.49,True,"{'id': '2f747304-1402-4d20-b7d5-727be0ee2d33',...",20,18,"{'context': 'DEFAULT', 'regionGroup': 'e1dbd9d...",BRL,"[{'dayOfWeek': 'SUNDAY', 'start': '11:00:00', ...",507.0,NaN,NaN,NaN,103.0,30.633981,17.417534,1.00,13.900,29.90,40.90,99.90
d7bbceb5-e10d-4228-a941-f18ab4ba3b27,Domino's Pizza - Parque Verde,"[{'type': 'LOGO', 'fileName': 'd7bbceb5-e10d-4...",belem-pa/dominos-pizza---parque-verde-parque-v...,4.63801,EXPENSIVE,"{'code': 'PIZ', 'name': 'Pizza'}","[DELIVERY, TAKEOUT, IMMEDIATE_ORDER]","[ADDRESS_PREFORM_TYPE, ALL_OFF_SET19, BF_PIZZA...","[DNR, MPAY, MOVPAY_MC, MC, GPY_ELO, REC, RDRES...",15.0,"{'type': 'FIXED', 'value': 4.0, 'originalValue...",35,0.74,True,"{'id': '0096d292-aa9d-44ab-93ea-15e21003d839',...",0,15,"{'context': 'DEFAULT', 'regionGroup': 'e1dbd9d...",BRL,"[{'dayOfWeek': 'SUNDAY', 'start': '16:00:00', ...",220.0,NaN,NaN,NaN,47.0,40.451064,30.680654,2.50,18.900,23.90,69.90,112.90
a9359f25-4660-4a54-b3b0-750df4c3569a,Spoleto - Parque Shopping Belém,"[{'type': 'LOGO', 'fileName': '202004141429_a9...",belem-pa/spoleto---parque-shopping-belem-parqu...,4.57014,MODERATE,"{'code': 'ITA', 'name': 'Italiana'}","[DELIVERY, TAKEOUT, TRACKING, TRACEABLE, IMMED...","[ADDRESS_PREFORM_TYPE, GUIDED_HELP_TYPE, INVER...","[DNR, MPAY, MOVPAY_MC, MC, GPY_ELO, ELOD, GPY_...",20.0,"{'type': 'FIXED', 'value': 6.99, 'originalValu...",40,0.38,True,"{'id': '6519d028-a268-4158-9ad3-8d0d92867b45',...",20,15,"{'context': 'DEFAULT', 'regionGroup': 'e1dbd9d...",BRL,"[{'dayOfWeek': 'SUNDAY', 'start': '11:00:00', ...",446.0,NaN,NaN,NaN,51.0,31.842941,14.126898,4.90,28.305,31.90,37.85,85.90
82e68657-6cd9-4f57-8360-2417526004c2,Subway - Grão Pará,"[{'type': 'LOGO', 'fileName': '201910231530_82...",belem-pa/subway---grao-para-parque-verde,4.41494,CHEAP,"{'code': 'LCH', 'name': 'Lanches'}","[DELIVERY, TAKEOUT, TRACKING, TRACEABLE, IMMED...","[ADDRESS_PREFORM_TYPE, CPGN_USER_DISCOUNT_5_LI...","[DNR, MPAY, MOVPAY_MC, MC, GPY_ELO, ELOD, GPY_...",15.0,"{'type': 'FIXED', 'value': 6.99, 'originalValu...",40,2.39,True,"{'id': '9bd5d0a5-8125-432f-809c-e99ee6284fd0',...",20,15,"{'context': 'DEFAULT', 'regionGroup': 'e1dbd9d...",BRL,"[{'dayOfWeek': 'SUNDAY', 'start': '12:00:00', ...",241.0,NaN,NaN,NaN,69.0,33.331884,15.698562,5.80,24.200,31.90,43.30,68.90
4b3160f1-8e8c-4a2b-a43e-e2962fa16df4,Mcdonalds Parque Shopping,"[{'type': 'LOGO', 'fileName': '202009111445_4b...",belem-pa/mcdonalds-parque-shopping-parque-verde,4.47222,CHEAPEST,"{'code': 'LCH', 'name': 'Lanches'}","[DELIVERY, TRACKING, TRACEABLE, IMMEDIATE_ORDER]","[20_30_OFF, ADDRESS_PREFORM_TYPE, BF_CUPOM15OF...","[DNR, MPAY, MOVPAY_MC, MC, GPY_ELO, ELOD, GPY_...",15.0,"{'type': 'FIXED', 'value': 7.99, 'originalValu...",34,0.58,True,"{'id': '78f6d594-aeb2-4a7d-8f4b-a9e56d23adff',...",20,10,"{'context': 'DEFAULT', 'regionGroup': 'e1dbd9d...",BRL,"[{'dayOfWeek': 'SUNDAY', 'start': '11:30:00', ...",398.0,NaN,NaN,NaN,93.0,25.302151,19.015800,2.90,14.900,19.60,37.80,134.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603df45a-b6fc-4040-9c5c-b1de46316757,Emporio Caseiro,"[{'type': 'LOGO', 'fileName': '6